# Imports

In [1]:
# clone my edit of caser
!git clone https://github.com/maxkvant/caser_pytorch

Cloning into 'caser_pytorch'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 84 (delta 2), reused 6 (delta 2), pack-reused 77
Unpacking objects: 100% (84/84), done.


In [1]:
import sys
import os
from collections import defaultdict
import numpy as np

sys.path.append('caser_pytorch')

from train_caser import Recommender
import argparse
from interactions import Interactions
from evaluation import evaluate_hits_ndcg

# Prepare SASRec datasets

In [8]:
!git clone https://github.com/kang205/SASRec

fatal: destination path 'SASRec' already exists and is not an empty directory.


In [16]:
# based on https://github.com/kang205/SASRec/blob/master/util.py

def data_partition(fname):
    usernum = 0
    itemnum = 0
    User = defaultdict(list)
    user_train = {}
    user_valid = {}
    user_test = {}
    # assume user/item index starting from 1
    with open(fname, 'r') as f:
        for line in f:
            u, i = line.rstrip().split(' ')
            u = int(u)
            i = int(i)
            usernum = max(u, usernum)
            itemnum = max(i, itemnum)
            User[u].append(i)

        for user in User:
            nfeedback = len(User[user])
            if nfeedback < 3:
                user_train[user] = User[user]
                user_valid[user] = []
                user_test[user] = []
            else:
                user_train[user] = User[user][:-2]
                user_valid[user] = []
                user_valid[user].append(User[user][-2])
                user_test[user] = []
                user_test[user].append(User[user][-1])
    return [user_train, user_valid, user_test, usernum, itemnum]

In [17]:
def save_to(path, train, test):
    try:
        os.makedirs(path)
    except:
        pass
    
    train_path = path + '/train.txt'
    test_path  = path + '/test.txt'
    
    with open(train_path, 'w') as f:
        for user, items in train.items():
            for item in items:
                print(f'{user} {item} 1', file=f)
    with open(test_path, 'w') as f:
        for user, items in test.items():
            for item in items:
                print(f'{user} {item} 1', file=f)

In [25]:
steam_train, _, steam_test, _, _ = data_partition('SASRec/data/Steam.txt')
save_to('data/Steam', steam_train, steam_test)

In [28]:
beauty_train, _, beauty_test, _, _ = data_partition('SASRec/data/Beauty.txt')
save_to('data/Beauty', beauty_train, beauty_test)

In [29]:
movies_train, _, movies_test, _, _ = data_partition('SASRec/data/ml-1m.txt')
save_to('data/ml-1m', movies_train, movies_test)

In [30]:
video_train, _, video_test, _, _ = data_partition('SASRec/data/Video.txt')
save_to('data/Video', video_train, video_test)

In [2]:
!ls -shn SASRec/data

total 57M
4.4M -rw-rw-r-- 1 1001 1003 4.4M Dec 14 14:03 Beauty.txt
4.0K -rw-rw-r-- 1 1001 1003 1.5K Dec 14 14:03 DataProcessing.py
 41M -rw-rw-r-- 1 1001 1003  41M Dec 14 14:03 Steam.txt
3.1M -rw-rw-r-- 1 1001 1003 3.1M Dec 14 14:03 Video.txt
8.7M -rw-rw-r-- 1 1001 1003 8.7M Dec 14 14:03 ml-1m.txt


In [3]:
!ls -shn data/*/

data/Beauty/:
total 4.5M
676K -rw-rw-r-- 1 1001 1003 675K Dec 14 14:22 test.txt
3.8M -rw-rw-r-- 1 1001 1003 3.8M Dec 14 14:22 train.txt

data/Steam/:
total 44M
4.5M -rw-rw-r-- 1 1001 1003 4.5M Dec 14 14:20 test.txt
 39M -rw-rw-r-- 1 1001 1003  39M Dec 14 14:20 train.txt

data/Video/:
total 3.2M
408K -rw-rw-r-- 1 1001 1003 406K Dec 14 14:23 test.txt
2.8M -rw-rw-r-- 1 1001 1003 2.8M Dec 14 14:23 train.txt

data/ml-1m/:
total 11M
68K -rw-rw-r-- 1 1001 1003 66K Dec 14 14:22 test.txt
11M -rw-rw-r-- 1 1001 1003 11M Dec 14 14:22 train.txt


In [18]:
train, _, test, _, _ = data_partition('beer_reviews.txt')
save_to('data/Beer', train, test)

In [19]:
!ls -shn data/*/

data/Beauty/:
total 4.5M
676K -rw-rw-r-- 1 1001 1003 675K Dec 14 14:22 test.txt
3.8M -rw-rw-r-- 1 1001 1003 3.8M Dec 14 14:22 train.txt

data/Beer/:
total 38M
300K -rw-rw-r-- 1 1001 1003 298K Dec 14 20:21 test.txt
 37M -rw-rw-r-- 1 1001 1003  37M Dec 14 20:21 train.txt

data/Steam/:
total 44M
4.5M -rw-rw-r-- 1 1001 1003 4.5M Dec 14 14:20 test.txt
 39M -rw-rw-r-- 1 1001 1003  39M Dec 14 14:20 train.txt

data/Video/:
total 3.2M
408K -rw-rw-r-- 1 1001 1003 406K Dec 14 14:23 test.txt
2.8M -rw-rw-r-- 1 1001 1003 2.8M Dec 14 14:23 train.txt

data/ml-1m/:
total 11M
68K -rw-rw-r-- 1 1001 1003 66K Dec 14 14:22 test.txt
11M -rw-rw-r-- 1 1001 1003 11M Dec 14 14:22 train.txt


# Intialize model

In [2]:
# based on https://github.com/graytowne/caser_pytorch/blob/master/train_caser.py

def get_model(n_iter=20):
    def str2bool(v):
        return v.lower() in ('true')
    
    parser = argparse.ArgumentParser()
    # data arguments
    parser.add_argument('--L', type=int, default=5)
    parser.add_argument('--T', type=int, default=3)
    # train arguments
    parser.add_argument('--n_iter', type=int, default=50)
    parser.add_argument('--seed', type=int, default=1234)
    parser.add_argument('--batch_size', type=int, default=512)
    parser.add_argument('--learning_rate', type=float, default=1e-3)
    parser.add_argument('--l2', type=float, default=1e-6)
    parser.add_argument('--neg_samples', type=int, default=3)
    parser.add_argument('--use_cuda', type=str2bool, default=True)

    config = parser.parse_args("")

    # model dependent arguments
    model_parser = argparse.ArgumentParser()
    model_parser.add_argument('--d', type=int, default=50)
    model_parser.add_argument('--nv', type=int, default=4)
    model_parser.add_argument('--nh', type=int, default=16)
    model_parser.add_argument('--drop', type=float, default=0.5)
    model_parser.add_argument('--ac_conv', type=str, default='relu')
    model_parser.add_argument('--ac_fc', type=str, default='relu')

    model_config = model_parser.parse_args("")
    model_config.L = config.L
    
    model = Recommender(n_iter=n_iter,
                        batch_size=config.batch_size,
                        learning_rate=config.learning_rate,
                        l2=config.l2,
                        neg_samples=config.neg_samples,
                        model_args=model_config,
                        use_cuda=config.use_cuda)
    return model

In [3]:
# based on https://github.com/graytowne/caser_pytorch/blob/master/train_caser.py

def read_dataset(path):
    train_path = path + '/train.txt'
    test_path  = path + '/test.txt'
    
    # load dataset
    train = Interactions(train_path)
    train.tocsr()
    
    # transform triplets to sequence representation
    train.to_sequence(5, 3)

    test = Interactions(test_path,
                        user_map=train.user_map,
                        item_map=train.item_map)
    test.tocsr()
    
    return train, test

# Movie lens

In [5]:
train, test = read_dataset('data/ml-1m')

model_movielens = get_model()
model_movielens.fit(train, test, verbose=True)

total training instances: 945251
Epoch 1 [34.0 s]	loss=0.8104 [0.0 s]
hits@10: 0.6478476821192053, ndcg@10: 0.3796063716863184
Epoch 2 [31.1 s]	loss=0.6064 [0.0 s]
Epoch 3 [31.2 s]	loss=0.5130 [0.0 s]
Epoch 4 [31.5 s]	loss=0.4586 [0.0 s]
Epoch 5 [31.4 s]	loss=0.4236 [0.0 s]
Epoch 6 [31.3 s]	loss=0.3993 [0.0 s]
Epoch 7 [31.5 s]	loss=0.3798 [0.0 s]
Epoch 8 [31.6 s]	loss=0.3644 [0.0 s]
Epoch 9 [31.7 s]	loss=0.3498 [0.0 s]
Epoch 10 [31.2 s]	loss=0.3375 [0.0 s]
hits@10: 0.7827814569536424, ndcg@10: 0.5416348625836132
Epoch 11 [31.4 s]	loss=0.3274 [0.0 s]
Epoch 12 [31.5 s]	loss=0.3179 [0.0 s]
Epoch 13 [31.5 s]	loss=0.3112 [0.0 s]
Epoch 14 [31.5 s]	loss=0.3038 [0.0 s]
Epoch 15 [31.3 s]	loss=0.2987 [0.0 s]
Epoch 16 [31.4 s]	loss=0.2943 [0.0 s]
Epoch 17 [31.5 s]	loss=0.2899 [0.0 s]
Epoch 18 [31.3 s]	loss=0.2862 [0.0 s]
Epoch 19 [31.0 s]	loss=0.2834 [0.0 s]
Epoch 20 [31.5 s]	loss=0.2807 [0.0 s]
hits@10: 0.7862582781456954, ndcg@10: 0.5518995905411003


In [6]:
%%time

evaluate_hits_ndcg(model_movielens, train, test)

CPU times: user 16.5 s, sys: 51.9 ms, total: 16.6 s
Wall time: 16.6 s


(0.7827814569536424, 0.5473805649720678)

# Video 

In [5]:
train, test = read_dataset('data/Video')

model_video = get_model()
model_video.fit(train, test, verbose=True)

total training instances: 101343
Epoch 1 [20.5 s]	loss=1.1125 [0.0 s]
hits@10: 0.43677814602792003, ndcg@10: 0.2514170970868496
Epoch 2 [3.4 s]	loss=0.9647 [0.0 s]
Epoch 3 [3.3 s]	loss=0.8753 [0.0 s]
Epoch 4 [3.3 s]	loss=0.7835 [0.0 s]
Epoch 5 [3.3 s]	loss=0.6947 [0.0 s]
Epoch 6 [20.5 s]	loss=0.6738 [0.0 s]
Epoch 7 [3.4 s]	loss=0.5945 [0.0 s]
Epoch 8 [3.3 s]	loss=0.5345 [0.0 s]
Epoch 9 [3.3 s]	loss=0.4863 [0.0 s]
Epoch 10 [3.6 s]	loss=0.4463 [0.0 s]
hits@10: 0.5351079859367153, ndcg@10: 0.3192788925597358
Epoch 11 [20.7 s]	loss=0.4504 [0.0 s]
Epoch 12 [3.4 s]	loss=0.4073 [0.0 s]
Epoch 13 [3.4 s]	loss=0.3730 [0.0 s]
Epoch 14 [3.4 s]	loss=0.3483 [0.0 s]
Epoch 15 [3.4 s]	loss=0.3228 [0.0 s]
Epoch 16 [20.7 s]	loss=0.3335 [0.0 s]
Epoch 17 [3.4 s]	loss=0.3052 [0.0 s]
Epoch 18 [3.3 s]	loss=0.2833 [0.0 s]
Epoch 19 [3.3 s]	loss=0.2644 [0.0 s]
Epoch 20 [3.4 s]	loss=0.2479 [0.0 s]
hits@10: 0.5670578794262213, ndcg@10: 0.35987679579369725


In [6]:
%%time

evaluate_hits_ndcg(model_video, train, test)

CPU times: user 26.4 s, sys: 60 ms, total: 26.4 s
Wall time: 26.4 s


(0.5762286860581746, 0.36362269353903226)

# Beauty 

In [4]:
train, test = read_dataset('data/Beauty')

model_beauty = get_model()
model_beauty.fit(train, test, verbose=True)

total training instances: 121810
Epoch 1 [61.4 s]	loss=1.1099 [0.0 s]
hits@10: 0.36121712222795255, ndcg@10: 0.20410765979649703
Epoch 2 [4.1 s]	loss=0.9602 [0.0 s]
Epoch 3 [4.1 s]	loss=0.8678 [0.0 s]
Epoch 4 [4.0 s]	loss=0.7771 [0.0 s]
Epoch 5 [4.2 s]	loss=0.7051 [0.0 s]
Epoch 6 [4.3 s]	loss=0.6401 [0.0 s]
Epoch 7 [4.2 s]	loss=0.5844 [0.0 s]
Epoch 8 [4.2 s]	loss=0.5388 [0.0 s]
Epoch 9 [4.1 s]	loss=0.4997 [0.0 s]
Epoch 10 [4.1 s]	loss=0.4680 [0.0 s]
hits@10: 0.37818257089629476, ndcg@10: 0.22407027085130296
Epoch 11 [4.3 s]	loss=0.4390 [0.0 s]
Epoch 12 [4.2 s]	loss=0.4115 [0.0 s]
Epoch 13 [4.1 s]	loss=0.3878 [0.0 s]
Epoch 14 [4.2 s]	loss=0.3670 [0.0 s]
Epoch 15 [4.2 s]	loss=0.3478 [0.0 s]
Epoch 16 [4.1 s]	loss=0.3306 [0.0 s]
Epoch 17 [4.1 s]	loss=0.3165 [0.0 s]
Epoch 18 [4.1 s]	loss=0.3024 [0.0 s]
Epoch 19 [4.1 s]	loss=0.2894 [0.0 s]
Epoch 20 [4.0 s]	loss=0.2787 [0.0 s]
hits@10: 0.3726298433635614, ndcg@10: 0.22878628717752122


In [6]:
train, test = read_dataset('data/Beauty')
evaluate_hits_ndcg(model_beauty, train, test)

(0.3642000826787929, 0.22389935923056878)

# Steam

In [7]:
train, test = read_dataset('data/Steam')

model_steam = get_model()

In [8]:
%%time

model_steam.fit(train, test, verbose=True)

total training instances: 1663064
Epoch 1 [186.5 s]	loss=0.7045 [0.0 s]
hits@10: 0.7458019105077929, ndcg@10: 0.4849447217200322
Epoch 2 [57.5 s]	loss=0.5799 [0.0 s]
Epoch 3 [57.6 s]	loss=0.5117 [0.0 s]
Epoch 4 [57.7 s]	loss=0.4623 [0.0 s]
Epoch 5 [58.0 s]	loss=0.4245 [0.0 s]
Epoch 6 [57.4 s]	loss=0.3952 [0.0 s]
Epoch 7 [57.3 s]	loss=0.3716 [0.0 s]
Epoch 8 [57.6 s]	loss=0.3538 [0.0 s]
Epoch 9 [57.5 s]	loss=0.3390 [0.0 s]
Epoch 10 [58.5 s]	loss=0.3273 [0.0 s]
hits@10: 0.7864175205947358, ndcg@10: 0.5319933167877577
Epoch 11 [57.5 s]	loss=0.3179 [0.0 s]
Epoch 12 [57.6 s]	loss=0.3105 [0.0 s]
Epoch 13 [57.7 s]	loss=0.3044 [0.0 s]
Epoch 14 [58.3 s]	loss=0.2991 [0.0 s]
Epoch 15 [57.8 s]	loss=0.2955 [0.0 s]
Epoch 16 [57.7 s]	loss=0.2915 [0.0 s]
Epoch 17 [57.7 s]	loss=0.2883 [0.0 s]
Epoch 18 [57.5 s]	loss=0.2861 [0.0 s]
Epoch 19 [58.0 s]	loss=0.2837 [0.0 s]
Epoch 20 [58.0 s]	loss=0.2815 [0.0 s]
hits@10: 0.8017302082285485, ndcg@10: 0.545679037760272
CPU times: user 22min 19s, sys: 22 s, total:

In [9]:
evaluate_hits_ndcg(model_steam, train, test)

(0.8060362173038229, 0.5476987024858936)

# Beer

In [20]:
train, test = read_dataset('data/Beer')

model_beer = get_model()

In [21]:
%%time

model_beer.fit(train, test, verbose=True)

total training instances: 2859902
Epoch 1 [141.5 s]	loss=0.3623 [0.0 s]
hits@10: 0.6108762210234728, ndcg@10: 0.3581718029171236
Epoch 2 [95.1 s]	loss=0.1348 [0.0 s]
Epoch 3 [95.7 s]	loss=0.1025 [0.0 s]
Epoch 4 [94.8 s]	loss=0.0935 [0.0 s]
Epoch 5 [94.8 s]	loss=0.0905 [0.0 s]
Epoch 6 [95.7 s]	loss=0.0891 [0.0 s]
Epoch 7 [95.2 s]	loss=0.0879 [0.0 s]
Epoch 8 [94.6 s]	loss=0.0874 [0.0 s]
Epoch 9 [95.8 s]	loss=0.0869 [0.0 s]
Epoch 10 [95.2 s]	loss=0.0864 [0.0 s]
hits@10: 0.6952479338842975, ndcg@10: 0.4628991585782418
Epoch 11 [94.8 s]	loss=0.0857 [0.0 s]
Epoch 12 [96.0 s]	loss=0.0853 [0.0 s]
Epoch 13 [95.2 s]	loss=0.0849 [0.0 s]
Epoch 14 [95.0 s]	loss=0.0849 [0.0 s]
Epoch 15 [95.7 s]	loss=0.0840 [0.0 s]
Epoch 16 [95.0 s]	loss=0.0840 [0.0 s]
Epoch 17 [96.1 s]	loss=0.0836 [0.0 s]
Epoch 18 [95.8 s]	loss=0.0834 [0.0 s]
Epoch 19 [95.1 s]	loss=0.0831 [0.0 s]
Epoch 20 [96.9 s]	loss=0.0829 [0.0 s]
hits@10: 0.7101789381050161, ndcg@10: 0.476839893008812
CPU times: user 32min 52s, sys: 32.3 s, tota